# Indexing Excercise

This exercise has two parts:

- In part 1, we are going to index the [MS MARCO](http://www.msmarco.org/) passage collection Pyserini toolkit and explore some features of the index. For this part, you only need to run code and understand it. You will be using the index and code snippets in the next assignment.

- In part 2, we are going to write a code for generating an inverted index and index part of MS MARCO collection. For this part, you need to first run the first part (1.1 and 1.2) to build the environment and prepare the data.




## PART 1: Generate the index via Pyserini

We use [Anserini](https://github.com/castorini/anserini]) toolkit and its python interface [Pyserini](https://github.com/castorini/pyserini)  to run our experiments.

***This part is created based on Anserini/Pyserini tutorials. You can learn more by checking their repositories and tutorials.*

### 1.1 Setup the environment

Install Pyserini:

In [4]:
!pip install pyserini

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.5/140.5 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 117.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 92.1 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 136.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.1 MB/s eta 0:00:00
  Created wheel for

Clone the Anserini repository from GitHub:

In [5]:
!git clone https://github.com/castorini/anserini.git
!cd anserini && git checkout ad5ba1c76196436f8a0e28efdb69960d4873efe3

Cloning into 'anserini'...
remote: Enumerating objects: 29354, done.
remote: Counting objects: 100% (3672/3672), done.
remote: Compressing objects: 100% (959/959), done.
remote: Total 29354 (delta 3146), reused 3059 (delta 2634), pack-reused 25682
Receiving objects: 100% (29354/29354), 85.32 MiB | 26.58 MiB/s, done.
Resolving deltas: 100% (19795/19795), done.
Note: switching to 'ad5ba1c76196436f8a0e28efdb69960d4873efe3'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at ad5ba1c7 Release notes for v0.9.2 (#1197)


### 1.2 Get the collection and prepare the files
MS MARCO (MicroSoft MAchine Reading COmprehension) is a large-scale dataset that defines many tasks from question answering to ranking. Here we focus on the collection designed for passage re-ranking.

In [6]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz -P data/msmarco_passage/

--2023-09-08 11:38:49--  https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1035009698 (987M) [application/octet-stream]
Saving to: ‘data/msmarco_passage/collection.tar.gz’

collection.tar.gz   100%[===================>] 987.06M  26.6MB/s    in 39s     

2023-09-08 11:39:29 (25.2 MB/s) - ‘data/msmarco_passage/collection.tar.gz’ saved [1035009698/1035009698]



In [7]:
!ls data/msmarco_passage/
!tar xvfz data/msmarco_passage/collection.tar.gz -C data/msmarco_passage

collection.tar.gz
collection.tsv


The original MS MARCO collection is a tab-separated values (TSV) file. We need to convert the collection into the jsonl format that can be processed by Anserini. jsonl files contain JSON object per line.

This command generates 9 jsonl files in your data/msmarco_passage/collection_jsonl directory, each with 1M lines (except for the last one, which should have 841,823 lines).

In [8]:
!cd anserini && python ./src/main/python/msmarco/convert_collection_to_jsonl.py \
 --collection_path ../data/msmarco_passage/collection.tsv --output_folder ../data/msmarco_passage/collection_jsonl


Converting collection...
Converted 0 docs in 1 files
Converted 100000 docs in 1 files
Converted 200000 docs in 1 files
Converted 300000 docs in 1 files
Converted 400000 docs in 1 files
Converted 500000 docs in 1 files
Converted 600000 docs in 1 files
Converted 700000 docs in 1 files
Converted 800000 docs in 1 files
Converted 900000 docs in 1 files
Converted 1000000 docs in 2 files
Converted 1100000 docs in 2 files
Converted 1200000 docs in 2 files
Converted 1300000 docs in 2 files
Converted 1400000 docs in 2 files
Converted 1500000 docs in 2 files
Converted 1600000 docs in 2 files
Converted 1700000 docs in 2 files
Converted 1800000 docs in 2 files
Converted 1900000 docs in 2 files
Converted 2000000 docs in 3 files
Converted 2100000 docs in 3 files
Converted 2200000 docs in 3 files
Converted 2300000 docs in 3 files
Converted 2400000 docs in 3 files
Converted 2500000 docs in 3 files
Converted 2600000 docs in 3 files
Converted 2700000 docs in 3 files
Converted 2800000 docs in 3 files
Conv

**Check the data!**

jsonl files are JSON files with keys id and contents:

In [9]:
!wc -l data/msmarco_passage/collection_jsonl/*

   1000000 data/msmarco_passage/collection_jsonl/docs00.json
   1000000 data/msmarco_passage/collection_jsonl/docs01.json
   1000000 data/msmarco_passage/collection_jsonl/docs02.json
   1000000 data/msmarco_passage/collection_jsonl/docs03.json
   1000000 data/msmarco_passage/collection_jsonl/docs04.json
   1000000 data/msmarco_passage/collection_jsonl/docs05.json
   1000000 data/msmarco_passage/collection_jsonl/docs06.json
   1000000 data/msmarco_passage/collection_jsonl/docs07.json
    841823 data/msmarco_passage/collection_jsonl/docs08.json
   8841823 total


In [10]:
!head -5 data/msmarco_passage/collection_jsonl/docs00.json

{"id": "0", "contents": "The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated."}
{"id": "1", "contents": "The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science."}
{"id": "2", "contents": "Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade."}
{"id": "3", "contents": "The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to t

Remove the original files to make room for the index.
Check the contents of `data/msmarco_passage` before and after.

In [11]:
!ls data/msmarco_passage
!rm data/msmarco_passage/*.tsv
!ls data/msmarco_passage
!rm -rf sample_data

collection_jsonl  collection.tar.gz  collection.tsv
collection_jsonl  collection.tar.gz


### 1.3 Generate the index using Pyserini


Here are some common indexing options with Pyserini (for more options, check Pyserini documentation):

```
* input: Path to collection
* threads: Number of threads to run
* collection: Type of Anserini Collection, e.g., LuceneDocumentGenerator, TweetGenerator (subclass of LuceneDocumentGenerator for TREC Microblog)
* index: Path to index output
* storePositions: Boolean flag to store positions
* storeDocvectors: Boolean flag to store document vectors
* storeRawDocs: Boolean flag to store raw document text
* keepStopwords: Boolean flag to keep stopwords (False by default)
* stemmer: Stemmer to use (Porter by default)
```

We now have everything in place to index the collection. **The indexing speed may vary, the process may take about 10 minutes (or more) in Google Colab.**




In [12]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 9 \
-input data/msmarco_passage/collection_jsonl -index indexes/lucene-index-msmarco-passage -storePositions -storeDocvectors -storeRaw

pyserini.index is deprecated, please use pyserini.index.lucene.
2023-09-08 11:43:06,993 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-09-08 11:43:06,996 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-09-08 11:43:06,996 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-09-08 11:43:06,997 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: data/msmarco_passage/collection_jsonl
2023-09-08 11:43:06,997 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-09-08 11:43:06,998 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-09-08 11:43:06,998 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 9
2023-09-08 11:43:06,999 INFO  [main] index.IndexCollection (IndexCollection.java:

Check the size of the index at the specified destination:

In [13]:
!ls indexes
!du -h indexes/lucene-index-msmarco-passage

lucene-index-msmarco-passage
4.1G	indexes/lucene-index-msmarco-passage


### 1.4 Explore Pyserini index

We can now explore the index using the The IndexReader class of Pyserini.

Read [Usage of the Index Reader API](https://github.com/castorini/pyserini/blob/master/docs/usage-indexreader.md) notebook for more information on accessing and manipulating an inverted index.

In [14]:
from pyserini.index import IndexReader

index_reader = IndexReader('indexes/lucene-index-msmarco-passage')

Compute the collection and document frequencies of a term:

In [15]:
term = 'played'

# Look up its document frequency (df) and collection frequency (cf).
# Note, we use the unanalyzed form:
df, cf = index_reader.get_term_counts(term)

analyzed_form = index_reader.analyze(term)
print(f'Analyzed form of term "{analyzed_form[0]}": df={df}, cf={cf}')

Analyzed form of term "plai": df=155044, cf=200696


Get basic index statistics of the index.

Note that unless the underlying index was built with the `-optimize` option (i.e., merging all index segments into a single segment), unique_terms will show -1 (think what could be reason).

In [16]:
index_reader.stats()

{'total_terms': 352316036,
 'documents': 8841823,
 'non_empty_documents': 8841823,
 'unique_terms': -1}

## PART 2: Generate the index yourself

### 2.1 Processing the text

We need to process the text, which includes tokenization, stopword removal, and lowercasing.

In [17]:
STOPWORDS = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'but', 'by', 'for', 'if', 'in', 'into', 'is', 'it', 'no', 'not', 'of', 'on', 'or', 'such', 'that', 'the', 'their', 'then', 'there', 'these', 'they', 'this', 'to', 'was', 'will', 'with']

def process(text):
    terms = []
    # Remove special characters
    chars = ['\'', '.', ':', ',', '!', '?', '(', ')']
    for ch in chars:
        if ch in text:
            text = text.replace(ch, ' ')

    # Lowercasing and stopword removal
    for term in text.split():
        term = term.lower()
        if term not in STOPWORDS:
            terms.append(term)
    return terms


### 2.2 Complete the code for Inverted Index

Implement the InvertedIndex class.

Write the index to a file, where posting list of each term is presented in a line with this format: `Term1 docID1:freq1 docID2:freq2 ...`, e.g.,

```
term1 1:1 4:2 5:1
term2 2:1
term3 1:3 3:3 9:2
...
```



In [28]:
class InvertedIndex(object):
    def __init__(self):
        self.index = {}

    def add_posting(self, term:str, doc_id:int, count:int):
        """Adds a posting (term and Document ID) to the index."""
        # =======Your code=======
        if term not in self.index:
          self.index[term] = []
        self.index[term].append({"doc_id":doc_id, "count":count})
        # =======================

    def get_posting(self,term:str):
        """Returns the posting list of the term from the index."""
        # =======Your code=======
        if term in self.index:
          return self.index[term]
        return None
        # =======================

    def get_dictionary(self):
        """Returns the dictionary of the index (unique terms in the index)."""
        # =======Your code=======
        return self.index
        # =======================

    def write_to_file(self, filename_index:str):
        """Writes the index to a textfile."""
        # =======Your code=======
        with open(filename_index,"w") as file:
          for key, values in self.index.items():
            posting = key
            for value in values:
              posting += f" {value['doc_id']}:{value['count']}"
            file.write(posting+'\n')
        # =======================

Run this to test your code. If everything is correct, you should not get errors here.

In [29]:
index = InvertedIndex()
index.add_posting("t1", 1, 2)
index.add_posting("t1", 2, 1)
index.add_posting("t2", 2, 3)
assert len(index.get_dictionary()) == 2
assert len(index.get_posting("t1")) == 2
assert index.get_posting("t3") == None
index.write_to_file("data/msmarco_passage/collection_jsonl/text_index.txt")

### 2.3 Index part of the MS MARCO collection

Complete the code to process the text and create the index.
Note that we are only interested in indexing `docs00.json` file and it takes few minutes to create the index.

In [30]:
import collections
import json

ind = InvertedIndex()
file = "data/msmarco_passage/collection_jsonl/docs00.json"
index_file = "data/msmarco_passage/collection_jsonl/tiny_index.txt"

def index(jsonl_file):
    with open(jsonl_file, 'r') as f:
        for line in f:
            doc = json.loads(line)
            # =======Your code=======
            processed_content = process(doc["contents"])
            # Get each term and make a dictionary counting the occurences of each term
            terms = {term:processed_content.count(term) for term in processed_content}
            for t,count in terms.items():
              ind.add_posting(t,doc["id"],count)
            # =======================

index(file)
ind.write_to_file(index_file)


Run this to test your code.

In [31]:
with open(index_file, 'r') as fp:
    assert len(fp.readlines()) == 698784

assert len(ind.get_posting("pressingly")) == 3
assert len(ind.get_posting("veada")) == 2

## Handing in

Hand in both the result file and the filled-in notebook:

- The result file should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_tiny_index.txt
- The notebook should be named STUDENTNUMBER_FIRSTNAME_LASTNAME_indexing.ipynb